In [2]:
import os
#import scipy.io
#import numpy as np
#import statistics, math
import matplotlib.pyplot as plt
#from libs import util_matlab as umatlab
#from libs import datasets, dataset_utils, utils
import tensorflow as tf

# We'll tell matplotlib to inline any drawn figures like so:
%matplotlib notebook
plt.style.use('ggplot')

In [12]:
# Number of sequences in a mini batch
batch_size = 100

# Number of characters in a sequence
sequence_length = 12

# Number of cells in our LSTM layer
n_cells = 256

# Number of LSTM layers
n_layers = 1

n_inputs=8

In [13]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, sequence_length, n_inputs], name='X')

# We'll have a placeholder for our true outputs
Y = tf.placeholder(tf.float32, [None, sequence_length], name='Y')

In [14]:
X.get_shape()

TensorShape([Dimension(None), Dimension(12), Dimension(8)])

In [15]:
if n_layers > 1:
    cells = tf.nn.rnn_cell.MultiRNNCell(
        [cells] * n_layers, state_is_tuple=True)
    initial_state = cells.zero_state(tf.shape(X)[0], tf.float32)
else:
    cells = tf.nn.rnn_cell.BasicLSTMCell(num_units=n_cells, state_is_tuple=True)
    initial_state = cells.zero_state(tf.shape(X)[0], tf.float32)

In [21]:
#tf.nn.rnn connects the Layer (variable cells) to the inputs (X) through tf.nn.rnn's second input parameter:
#tf.nn.rnn(cells, inputs=X, initial_state=initial_state)
#BUT parameter 'inputs=X' should be a list!
#So, convert X to a list:
print(X.get_shape().as_list())
# Permuting batch_size and n_steps. TO: (sequence_length, batch_size, n_inputs)
Xs = tf.transpose(X, [1, 0, 2])
print(Xs.get_shape().as_list())
# Reshaping to (sequence_length*batch_size, n_inputs)
Xs = tf.reshape(Xs, [-1, n_inputs])
print(Xs.get_shape().as_list())
# Split to get a list of 'sequence_length' tensors of shape (batch_size, n_input)
Xs = tf.split(0, sequence_length, Xs)
print(Xs)

[None, 12, 8]
[12, None, 8]
[None, 8]
[<tf.Tensor 'split_2:0' shape=(?, 8) dtype=float32>, <tf.Tensor 'split_2:1' shape=(?, 8) dtype=float32>, <tf.Tensor 'split_2:2' shape=(?, 8) dtype=float32>, <tf.Tensor 'split_2:3' shape=(?, 8) dtype=float32>, <tf.Tensor 'split_2:4' shape=(?, 8) dtype=float32>, <tf.Tensor 'split_2:5' shape=(?, 8) dtype=float32>, <tf.Tensor 'split_2:6' shape=(?, 8) dtype=float32>, <tf.Tensor 'split_2:7' shape=(?, 8) dtype=float32>, <tf.Tensor 'split_2:8' shape=(?, 8) dtype=float32>, <tf.Tensor 'split_2:9' shape=(?, 8) dtype=float32>, <tf.Tensor 'split_2:10' shape=(?, 8) dtype=float32>, <tf.Tensor 'split_2:11' shape=(?, 8) dtype=float32>]


In [19]:
Xs

[<tf.Tensor 'split:0' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'split:1' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'split:2' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'split:3' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'split:4' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'split:5' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'split:6' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'split:7' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'split:8' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'split:9' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'split:10' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'split:11' shape=(?, 8) dtype=float32>]

In [ ]:
# this will return us a list of outputs of every element in our sequence.
# Each output is `batch_size` x `n_cells` of output.
# It will also return the state as a tuple of the n_cells's memory and
# their output to connect to the time we use the recurrent layer.
outputs, state = tf.nn.rnn(cells, Xs, initial_state=initial_state)

In [ ]:
print(outputs)

In [ ]:
# We'll now stack all our outputs for every cell
outputs_flat = tf.reshape(tf.concat(1, outputs), [-1, n_cells])